In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)
import os
import sys
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one level
sys.path.append(project_path)
import pandas as pd
from numpy.random import random
from utils.chatbot import QWENChatBot
from pandarallel import pandarallel
import json
import time
import random
from collections import Counter
from concurrent.futures import ThreadPoolExecutor



INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
data_tagged = pd.read_excel('../processed_data/02_tagged_processed.xlsx')
data_tagged

,内容,raw_tagging,parsed_json,Age_Range,Gender,City_Tier,Purchase_Purpose,Purchase_Situation,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Keywords_Other,Emotion,Past_Purchase,Tone
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['联名快闪店', 'lv和村上隆']",NaN,否,活泼俏皮
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""...","{'Age_Range': '25-34岁', 'Gender': '女性', 'City_...",25-34岁,女性,一线,个人喜好/收藏,NaN,['LV水貂帽衫背心'],"['miumiu', 'Chanel']",['水貂帽衫背心'],NaN,NaN,"['LV村上隆', '北京skp店', 'vic客户体验']",喜欢,是,成熟稳重
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,打卡/拍照,NaN,NaN,NaN,NaN,"['黑白三彩店', '咖啡店']",NaN,"['快闪店', 'LV村上隆']",喜欢,否,活泼俏皮
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"['黑三彩Carryall', '黑武士羊皮carryall']",NaN,"['黑三彩', '黑武士羊皮']",NaN,"['单肩斜挎', '不压身高不显大']",NaN,喜欢,是,成熟稳重
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n...","{'Age_Range': '', 'Gender': '女性', 'City_Tier':...",NaN,女性,NaN,追求独特设计,NaN,"['卡包', '钱包']",NaN,"['白三彩', '黑三彩']",['可爱'],['实用'],NaN,喜欢,否,活泼俏皮
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,陪伴购物,逛街,['trainer'],NaN,NaN,NaN,NaN,"['母上大人', '蹭到', '心动']",开心,否,活泼俏皮
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"['村上隆太阳花熊猫', '手袋']",NaN,"['太阳花', '熊猫']",['合适'],NaN,['花花'],喜欢,否,活泼俏皮
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['快闪店', '咖啡店', '出片', '跨年', 'ccd拍照']",满意,否,活泼俏皮
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"['speedy系列', 'speedynano']",NaN,"['speedy30小', '上身偏大']",['酷'],NaN,['oversize'],满意,否,成熟稳重


In [4]:
import ast
col_list = ['Product_Mention',
       'Competitor', 'Keywords_Design_Elements', 'Keywords_Design_Style',
       'Keywords_Functionality', 'Keywords_Other']
for col in col_list:

# Convert string representations of lists to actual Python lists
    data_tagged[col] = data_tagged[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

In [5]:
#Purpose clustering
from typing import List
system_message = """
你是一名专业的数据标注专家，我为你准备了一些用户的购买需求，目标是对这些需求进行分类，并为每条需求匹配1个最 relevant 的标签。标签需从以下 8 个类别中选择：

已知标签：
1.实用性: 用户对产品功能性、日常使用场景、便捷性需求的描述。
关键词参考：容量、耐用、收纳方便、提升体验、实用性强、日常需求、通勤需要。
2.设计感: 用户对产品美观、独特设计、细节设计的关注，或对奢侈优雅设计的追求。
关键词参考：外观设计、审美偏好、独特、时尚、精致、美感、风格搭配。
3.潮流: 用户因时尚潮流趋势、新品流行款、爆款等受到吸引并关注。
关键词参考：潮流趋势、热门单品、当季新品、明星同款、追逐潮流、联名款、时尚潮搭。
4.复古情怀: 用户关注与怀旧、记忆、传统经典相关的产品与设计。
关键词参考：怀旧、经典设计、复刻款、情感寄托、复古风、青春回忆。
5.品牌情怀: 用户因品牌忠诚度、对品牌文化或艺术联名的喜爱而关注购买。
关键词参考：品牌认同、联名合作款、艺术感、品牌价值、品牌魅力、支持设计师。
6.社交需求: 用户希望通过产品实现社交互动、传播展示或内容分享的需求。
关键词参考：打卡、分享、晒图、炫耀、社交媒体、曝光率、社交互动、拍照需求。
7.收藏&投资: 用户为实现收藏、长期保存或投资价值而购买。
关键词参考：稀有、限量版、经典收藏、未来升值、投资价值、收藏情怀、长期持有。
8.其他: 无法归属到以上具体类别的需求。
关键词参考：模糊需求、不清楚的描述、仅提到但无明确购买动机。



标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。
4.如果评论语义不清晰、无法提取主要购买动机，请返回标签为 其他。

输出要求：
不要添加任何解释、注释或多余内容，例如"\n\n设计感"。
严格按照规则返回以上8个标签名称.



"""
chatbot = QWENChatBot(system_message=system_message)


# 使用线程池处理 DataFrame 数据
def classify_post(content: str) -> str:
    """
    A process-safe function to classify a single post using the chatbot.
    """
    try:
        if pd.isnull(content):  # Handle NaN (null) input
            return "未明确提及"
        return chatbot.ask(content)  # Ensure this returns only the classification
    except Exception as e:
        return f"Error: {e}"  # Return error string
data_tagged['Purpose_Category'] = data_tagged['Purchase_Purpose'].parallel_apply(lambda x:classify_post(x))

#data_tagged['Purpose_Category'] = classify_posts_concurrently(data_tagged['Purchase_Purpose'].tolist(), max_workers=16)
data_tagged['Purpose_Category'].value_counts()


Purpose_Category
未明确提及    981
设计感      938
其他       230
实用性      156
潮流        95
复古情怀      89
收藏&投资     85
社交需求      29
品牌情怀      24
Name: count, dtype: int64

In [6]:
#Situation clustering
from typing import List
system_message = """
你是一名数据标注专家，现在需要对用户的购买LVMH和村上隆的场景进行归类，例如通勤，逛街，穿搭，旅游，节日等，并为每条需求匹配1个最 relevant 的标签。

已知存在以下几个类别：
1.通勤: 涉及日常上下班、工作或学习等场景。
2.逛街: 涉及休闲购物、逛商场或随手试用等情境。
3.穿搭: 与个人时尚协调、提升造型相关的穿戴或搭配情境。
4.旅游: 涉及旅行、度假或日常出行的场景。
5.节日: 如新年、生日、春节等与节庆活动相关的情境。
6.日常使用: 其他日常场景，如居家、日常生活
7.其他: 无法归入以上几类的


标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。


输出要求：
不要添加任何解释、注释或多余内容.
严格按照规则返回以上7个标签名称.

要求：
不要添加任何解释、注释或多余内容。
严格按照规则返回标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)

# 使用线程池处理 DataFrame 数据
data_tagged['Situation_Category'] = data_tagged['Purchase_Situation'].parallel_apply(lambda x:classify_post(x))
data_tagged['Situation_Category'].value_counts()

Situation_Category
未明确提及    1669
逛街        344
日常使用      218
穿搭        120
节日        109
旅游         86
其他         52
通勤         29
Name: count, dtype: int64

In [10]:
#Emotion clustering
system_message = """
你是一名专业的数据标注专家，我为你准备了一些用户对产品的情绪，目标是对这些情绪进行分类，并为每条需求匹配1个最 relevant 的标签。标签必须从以下几个类别中选择：

喜欢/期待/惊喜/失望/无感/满意/不满/中规中矩/想要



标注规则：
1.单一标签：每条情绪只能分配 1 个标签，不得多选或模糊。

输出要求：
不要添加任何解释、注释或多余内容，例如“喜欢\n喜欢”。
严格按照规则返回以上标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)
data_tagged['Emotion_Category'] = data_tagged['Emotion'].parallel_apply(lambda x:classify_post(x))
data_tagged['Emotion_Category'].value_counts()

Emotion_Category
喜欢                                                                                                                                                                                                                      1339
未明确提及                                                                                                                                                                                                                    462
满意                                                                                                                                                                                                                       242
无感                                                                                                                                                                                                                       222
失望                                                                                                 

In [13]:
# Replace cell values in emotion where length >= 6 with a specific value
data_tagged['Emotion_Category'] = data_tagged['Emotion_Category'].apply(lambda x: '未明确提及' if len(x) >= 6 else x)
data_tagged['Emotion_Category'].value_counts()

Emotion_Category
喜欢       1339
未明确提及     465
满意        242
无感        222
失望        137
不满         96
惊喜         52
期待         49
中规中矩       18
想要          7
Name: count, dtype: int64

In [23]:
#Design style clustering
system_message = """
你是一名数据标注专家，现在需要对用户的lvmh新品设计风格反馈进行归类，为每条风格匹配1个最 relevant 的标签。

可爱/时尚/复古/简洁/好看/高级感/千禧年/独特/百搭/其他

标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。


输出要求：
不要添加任何解释、注释或新增标签.
严格按照规则返回以上标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)

# Function to classify a single list of strings
def classify_post_list(content_list) -> list:
    """
    A process-safe function to classify a list of strings using the chatbot.
    Handles single strings as well as lists of strings.
    """
    try:
        # Handle NaN or invalid (non-list) input
        if len(content_list) == 0:  # Handle null values explicitly
            return ["未明确提及"]

        # If the content_list contains a single string (not a list), process it directly
        if isinstance(content_list, str):
            time.sleep(random.uniform(0.2, 0.8))
            return [chatbot.ask(content_list)]  # Return a single classification as a list

        # If content_list is a list of strings, classify each string in the list
        if isinstance(content_list, list):
            time.sleep(random.uniform(0.2, 0.8))
            return list(set([chatbot.ask(content) for content in content_list]))

        # If input is not a string or list, return invalid input
        return ["无效输入"]  # Handle unexpected formats gracefully
    except Exception as e:
        return [f"Error: {e}"]  # Return an error as a list for consistency

# Apply the function to the DataFrame column
data_tagged['Style_Category'] = data_tagged['Keywords_Design_Style'].parallel_apply(lambda x:classify_post_list(x) )
data_tagged['Style_Category'].value_counts()

Style_Category
[未明确提及]              1383
[可爱]                  225
[好看]                  160
[时尚]                  125
[复古]                   87
                     ... 
[好看, 简洁]                1
[好看, 高级感, 时尚]           1
[其他, 简洁, 可爱, 高级感]       1
[千禧年, 复古, 独特]           1
[百搭, 高级感, 时尚]           1
Name: count, Length: 123, dtype: int64

In [24]:
Counter([j for i in data_tagged['Style_Category'] for j in i])

Counter({'未明确提及': 1383,
         '可爱': 399,
         '时尚': 298,
         '好看': 262,
         '复古': 219,
         '独特': 159,
         '高级感': 156,
         '简洁': 97,
         '百搭': 81,
         '其他': 67,
         '千禧年': 60,
         '温暖感': 1})

In [25]:
#functionality clustering
system_message = """
你是一名数据标注专家，现在需要对lvmh新品设计功能性进行归类，匹配1个最 relevant 的标签。

斜挎/手提/单肩/实用/大容量/日常/小巧/多功能/自重轻/其他

标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。


要求：
不要添加任何解释、注释或多余内容.
严格按照规则返回以上标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)
data_tagged['Functionality_Category'] = data_tagged['Keywords_Functionality'].parallel_apply(lambda x:classify_post_list(x) )
Counter([j for i in data_tagged['Functionality_Category'] for j in i])

Counter({'未明确提及': 2080,
         '其他': 198,
         '实用': 155,
         '大容量': 144,
         '日常': 79,
         '多功能': 70,
         '小巧': 66,
         '斜挎': 64,
         '手提': 52,
         '自重轻': 39,
         '单肩': 31})

In [26]:
#elements clustering

system_message = """
你是一名数据标注专家，根据你对lvmh产品的了解，现在需要对lvmh新品设计元素进行归类，匹配1个最 relevant 的标签。

已知有如下几个标签：
黑三彩/白三彩/老花/太阳花/黑色/棕色/白色/马卡龙色/蝴蝶结/三彩/樱桃/樱花/其他

标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。


要求：
不要添加任何解释、注释或多余内容.
严格按照规则返回以上几类标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)
data_tagged['Element_Category'] = data_tagged['Keywords_Design_Elements'].parallel_apply(lambda x:classify_post_list(x) )
Counter([j for i in data_tagged['Element_Category'] for j in i])

Counter({'未明确提及': 1339,
         '其他': 743,
         '白三彩': 345,
         '白色': 143,
         '三彩': 120,
         '老花': 118,
         '黑三彩': 112,
         '太阳花': 104,
         '马卡龙色': 103,
         '黑色': 96,
         '蝴蝶结': 69,
         '樱花': 16,
         '樱桃': 13,
         '棕色': 12,
         '蓝色': 3})

In [27]:
#product clustering
system_message = """
你是一名数据标注专家，根据你对lvmh产品的了解，现在需要对用户提到的lvmh产品进行归类，匹配1个最 relevant 的标签。

已知有如下几个标签：
speedy/carryall//all in bb/side trunk/neverfull/capucines/suitcase/small leather goods/accessorries/其他

标注规则：
1.单一标签：每条需求只能分配 1 个标签，不得多选或模糊。
2.优先级判断：如果需求涉及多个标签，请根据需求表达的核心内容，选择最符合的标签。
3.Small leather goods:包括麻将包/零钱包/护照夹等小皮具
4.accessories： 包括丝巾/帽子/腰带/眼镜等配饰



示例1 
输入为'speedy黑三彩','lvspeedy','Speedy新款'等内容，你需要返回'speedy'
示例2
输入为'太阳眼镜','丝巾'等内容，你需要返回'accessories'


要求：
不要添加任何解释、注释或多余内容.
严格按照规则返回以上几类标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)
data_tagged['Product_Category'] = data_tagged['Product_Mention'].parallel_apply(lambda x:classify_post_list(x) )
Counter([j for i in data_tagged['Product_Category'] for j in i])

Counter({'speedy': 900,
         '未明确提及': 695,
         '其他': 667,
         'small leather goods': 334,
         'accessories': 322,
         'carryall': 294,
         'side trunk': 113,
         'all in bb': 104,
         'neverfull': 54,
         'other': 52,
         'capucines': 50,
         'accessorries': 25,
         'suitcase': 11})

In [28]:
#Competitor clustering
system_message = """
你是一名数据标注专家，根据你对奢侈品牌的了解，现在需要对用户提到的除lv以外的品牌进行归类，匹配1个最 relevant 的标签。

标注规则：  
1.单一标签：每条输入只能分配 1 个标签，返回全部大写的官方品牌名称
2.如果无法匹配的，返回'未明确提及'

示例1 
输入为'hermes','爱马仕','Hermes'等内容，你需要返回'HERMES'
示例2
输入为'Miumiu','miu miu','Miumiu'等内容，你需要返回'MIUMIU'


要求：
不要添加任何解释、注释或多余内容.
严格按照规则返回以上几类标签名称.
"""
chatbot = QWENChatBot(system_message=system_message)
data_tagged['Competitor_Category'] = data_tagged['Competitor'].parallel_apply(lambda x:classify_post_list(x) )
Counter([j for i in data_tagged['Competitor_Category'] for j in i])

Counter({'未明确提及': 2488,
         'CHANEL': 50,
         'HERMES': 26,
         'DIOR': 18,
         'MIUMIU': 15,
         'PRADA': 10,
         'GUCCI': 8,
         'BURBERRY': 6,
         'BALENCIAGA': 6,
         'LOEWE': 5,
         'CELINE': 5,
         'COACH': 5,
         'BVLGARI': 5,
         'CARTIER': 5,
         'CHROME HEARTS': 5,
         'YSL': 4,
         'BAPE': 4,
         'FENDI': 4,
         'ACNE STUDIOS': 4,
         'SKIMS': 3,
         'VIVIENNE WESTWOOD': 3,
         'SUPREME': 3,
         'MCM': 3,
         'VALENTINO': 3,
         'CANADA GOOSE': 3,
         'DELVAUX': 2,
         'JIMMY CHOO': 2,
         'LULULEMON': 2,
         'DIPTYQUE': 2,
         'BOTTEGA VENETA': 2,
         'ROGER VIVIER': 2,
         'BALMAIN': 2,
         'DIESEL': 2,
         'TIFFANY': 2,
         'MONCLER': 2,
         'ALEXANDER MCQUEEN': 1,
         'JEAN-CHARLES DE CASTELBAJAC': 1,
         'MICHAEL KORS': 1,
         'LA MER': 1,
         'PATEK PHILIPPE': 1,
         'SALV

In [30]:
data_tagged

,内容,raw_tagging,parsed_json,Age_Range,Gender,City_Tier,Purchase_Purpose,Purchase_Situation,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Keywords_Other,Emotion,Past_Purchase,Tone,Purpose_Category,Situation_Category,Emotion_Category,Style_Category,Functionality_Category,Element_Category,Product_Category,Competitor_Category
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pancake，班尼迪还有椰奶燕麦粥都很好吃！就是店不太大可能需要排会儿队。结账的时候看到有开心果可颂🥐，准备下次试试 2️⃣喜茶最近的草间弥生联名，伤心了自从英区喜茶没有脆波波之后感觉喝的频率少了好多 3️⃣无意间买到了南瓜labubu🎃！是牛津街的泡泡玛特，就是没看到小春野和可乐 4️⃣5️⃣ 伦敦DR💆🏻‍♀️ 这几天脸上爆痘太多，放学后去市中心金融城附近尝试了一下深层清洁，Hydra主打调理皮肤，把皮肤深层的脏东西吸出去。做Hydra前会先用专业仪器做visia分析我的肤况。有听不懂的也没关系有中文team可以翻译。整个过程也很舒服，黑头基本都没了，感觉肤色也提亮了不少。小姐姐还很贴心的告知第二天就可以正常护肤啦～皮肤干净清爽后，下次准备再去做其他的，还种草了一些光电类，准备慢慢尝试。 6️⃣某天放学～喜欢看伦敦的落日🌇 7️⃣8️⃣也是放学闲来无事和朋友去看lv和村上隆的联名快闪店，一楼卖东西二楼是咖啡店，好多好多人 9️⃣holborn地铁站的告示牌～经常可以看到工作人员的创作哈哈哈哈！好可爱～ #留学英国 #发点生活边角料 #清清日常库存 #留学plog #伦敦brunch #labubu,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""女性"",\n ""City_Tier"": """",\n ""Purchase_Purpose"": """",\n ""Purchase_Situation"": """",\n ""Product_Mention"": [],\n ""Competitor"": [],\n ""Emotion"": """",\n ""Tone"": ""活泼俏皮"",\n ""Past_Purchase"": ""否"",\n ""Keywords"": {\n ""Design_Elements"": [],\n ""Design_Style"": [],\n ""Functionality"": [],\n ""Other"": [""联名快闪店"", ""lv和村上隆""]\n }\n}","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_Tier': '', 'Purchase_Purpose': '', 'Purchase_Situation': '', 'Product_Mention': [], 'Competitor': [], 'Emotion': '', 'Tone': '活泼俏皮', 'Past_Purchase': '否', 'Keywords': {'Design_Elements': [], 'Design_Style': [], 'Functionality': [], 'Other': ['联名快闪店', 'lv和村上隆']}}",18-24岁,女性,NaN,NaN,NaN,[],[],[],[],[],"[联名快闪店, lv和村上隆]",NaN,否,活泼俏皮,未明确提及,未明确提及,未明确提及,[未明确提及],[未明确提及],[未明确提及],[未明确提及],[未明确提及]
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下图了，先挑最爱的来吧！ \t 首先来分享一波LV吧，都是在北京skp店买的，一眼看中了一件水貂帽衫背心特别喜欢，他们店的vic客户体验绝对是超有含金量了。一落地北京，酒店房间就被投喂了我一直想买都没有买到的巧克力酱和巧克力小礼盒。下午到了6楼的LV lounge还有我的水彩画和各式礼物，还安排了带我们体LV花茶包制作以及竖琴欣赏。晚餐也安排的很美味！购物起来也无压力不会推销全程愉悦，越买越愉悦[汗颜R]Sa很给力 \t 接下来是miumiu系列，羽绒服和厚底羊羔毛靴都是为了去日本滑雪准备～还搭配了很可爱的发夹一套拍照[自拍R]卡其色的西装是年末5折买的；还有25春夏tee棉质柔软～ \t 重头戏！护肤品方面最近入手了两款还蛮推荐的！ 第一款是贝熙迦儿的龙舌兰眼霜，这是被我的美容师安利的单品，她跟我说众多好莱坞明星维密超模的挚爱～即时效果提拉紧致祛水肿功效特别的好！用了半个月体验感还是蛮不错的，乳液质地但是很好吸收，很适合秋冬，也没有长脂肪粒用下来，眼下泪沟倒是用下来真的有点淡了，早上起来微微肿的眼皮也消肿快了不少，不过还是要耐心按摩几下的哈，这样效果更好[加油R] 第二款就是Chanel奢华精萃修护精华油，价格真的够奢华的[捂脸R]一开始我都不太舍得打开用，是闺蜜送的！直到前段时间我的脸太干了才想说打开试试，第二天早晨是真的又润又亮气色又好。滋养力非常强悍，推荐有实力的宝宝入坑。 \t Chanel最近买了两个蛮可爱的同色系小包，一个是双肩背的，一个是斜背的，都是灰色系。蓝色格纹裙改了一下腰围拿回来了～ \t 最后再分享我最近特别喜欢的品牌乘光LUMIHOOD的牡丹沉香底座，太美了！！每一朵花都栩栩如生，听说都是纯手工的，每天只能做5朵成品而已～ \t 以上就是农历新年之前我的开箱分享啦，祝愿大家己巳年日有熹光，欣喜在望，巳全巳美。去追求自己所热爱的，去拼搏自己所向往的。绽放自己的美与欲望，坦然花钱，努力赚钱[鼓掌R] \t #购物开箱分享 #LV #lv村上隆 #miumiu#chanel #新年礼物 #乘光lumihood #贝熙迦儿 #眼霜分享 #精华油推荐,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""女性"",\n ""City_Tier"": ""一线"",\n ""Purchase_Purpose"": ""个人喜好/收藏"",\n ""Purchase_Situation"": """",\n ""Product_Mention"": [""LV水貂帽衫背心""],\n ""Competitor"": [""miumiu"", ""Chanel""],\n ""Emotion"": ""喜欢"",\n ""Tone"": ""成熟稳重"",\n ""Past_Purchase"": ""是"",\n ""Keywords"": {\n ""Design_Elements"": [""水貂帽衫背心""],\n ""Design_Style"": [],\n ""Functionality"": [],\n ""Other"": [""LV村上隆"", ""北京skp店"", ""vic客户体验""]\n }\n}","{'Age_Range': '25-34岁', 'Gender': '女性', 'City_Tier': '一线', 'Purchase_Purpose': '个人喜好/收藏', 'Purchase_Situation': '', 'Product_Mention': ['LV水貂帽衫背心'], 'Competitor': ['miumiu', 'Chanel'], 'Emotion': '喜欢', 'Tone': '成熟稳重', 'Past_Purchase': '是', 'Keywords': {'Design_Elements': ['水貂帽衫背心'], 'Design_Style': [], 'Functionality': [], 'Other': ['LV村上隆', '北京skp店', 'vic客户体验']}}",25-34岁,女性,一线,个人喜好/收藏,NaN,[LV水貂帽衫背心],"[miumiu, Chanel]",[水貂帽衫背心],[],[],"[LV村上隆, 北京skp店, vic客户体验]",喜欢,是,成熟稳重,收藏&投资,未明确提及,喜欢,[未明确提及],[未明确提及],[其他],[accessories],"[MIUMIU, CHANEL]"
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）我还特地一早去的（虽然到的时候也11点多了）咖啡店排到了对面街的红绿灯处。保安说最少排一个小时，当即放弃了 hhhh 那就拍拍照吧！还是挺出片的！我觉得黑白三彩店最好拍～咖啡店可能真的得进去才能拍好看吧 这次的pop up时间真的太短了！才一周！lv是懂快闪的[笑哭R]#时尚潮人打卡地 #随处走走就是潮流 #潮流新地标,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""女性"",\n ""City_Tier"": 

In [42]:
data_tagged.to_excel('../processed_data/03_tagged_processed_cleaned.xlsx',index=False)

In [38]:
for group in data_tagged['Age_Range'].unique():
    temp = data_tagged[data_tagged['Age_Range'] == group]['Product_Category']
    l = Counter([j for i in temp for j in i if j !='未明确提及' and j!='其他'])
    print(group,l)

18-24岁 Counter({'speedy': 52, 'accessories': 31, 'small leather goods': 25, 'carryall': 16, 'side trunk': 10, 'all in bb': 10, 'capucines': 4, 'other': 4, 'neverfull': 3})
25-34岁 Counter({'speedy': 174, 'accessories': 64, 'carryall': 60, 'small leather goods': 45, 'all in bb': 26, 'side trunk': 21, 'neverfull': 12, 'other': 12, 'capucines': 8, 'suitcase': 4, 'accessorries': 4})
nan Counter()
35-44岁 Counter({'speedy': 64, 'small leather goods': 21, 'carryall': 17, 'accessories': 15, 'neverfull': 6, 'side trunk': 4, 'other': 3, 'capucines': 3, 'all in bb': 2, 'accessorries': 2})
45岁以上 Counter({'speedy': 12, 'small leather goods': 5, 'accessories': 3, 'capucines': 1, 'neverfull': 1, 'carryall': 1, 'all in bb': 1})
18岁以下 Counter({'carryall': 1})


In [39]:
for group in data_tagged['Age_Range'].unique():
    temp = data_tagged[data_tagged['Age_Range'] == group]['Style_Category']
    l = Counter([j for i in temp for j in i if j !='未明确提及' ])
    print(group,l)

    

18-24岁 Counter({'可爱': 59, '时尚': 38, '好看': 33, '高级感': 19, '复古': 15, '百搭': 12, '独特': 12, '简洁': 10, '其他': 9, '千禧年': 6})
25-34岁 Counter({'复古': 81, '可爱': 76, '时尚': 64, '独特': 35, '好看': 33, '高级感': 31, '简洁': 28, '千禧年': 27, '百搭': 20, '其他': 17, '温暖感': 1})
nan Counter()
35-44岁 Counter({'复古': 26, '可爱': 23, '时尚': 22, '好看': 11, '千禧年': 9, '高级感': 9, '简洁': 6, '独特': 5, '其他': 5, '百搭': 1})
45岁以上 Counter({'好看': 2, '简洁': 2, '复古': 1, '独特': 1, '时尚': 1, '可爱': 1})
18岁以下 Counter({'其他': 1, '高级感': 1})


In [77]:
for group in data_tagged['Age_Range'].unique():
    temp = data_tagged[data_tagged['Age_Range'] == group]['Purpose_Category']
    l = Counter([j for j in temp if j !='未明确提及'])
    print(group,l)


nan Counter()
25-34岁 Counter({'设计感': 55, '复古情怀': 10, '其他': 7, '实用性': 4, '品牌情怀': 3, '收藏&投资': 3, '潮流': 2})
18-24岁 Counter({'设计感': 49, '其他': 4, '复古情怀': 3, '实用性': 2, '潮流': 1, '收藏&投资': 1})
35-44岁 Counter({'设计感': 11, '复古情怀': 8, '其他': 4, '潮流': 1})
45岁以上 Counter({'设计感': 5, '其他': 3, '实用性': 3, '复古情怀': 3, '品牌情怀': 2, '收藏&投资': 1})
18岁以下 Counter({'设计感': 1})


In [78]:
for group in data_tagged['Age_Range'].unique():
    temp = data_tagged[data_tagged['Age_Range'] == group]['Situation_Category']
    l = Counter([j for j in temp if j !='未明确提及'])
    print(group,l)

nan Counter()
25-34岁 Counter({'日常使用': 25, '逛街': 16, '节日': 10, '通勤': 7, '穿搭': 5, '旅游': 4, '其他': 1})
18-24岁 Counter({'穿搭': 14, '日常使用': 12, '逛街': 6, '节日': 5, '旅游': 3, '其他': 1})
35-44岁 Counter({'日常使用': 9, '节日': 3, '通勤': 1, '穿搭': 1})
45岁以上 Counter({'日常使用': 5, '逛街': 1, '节日': 1, '穿搭': 1, '旅游': 1})
18岁以下 Counter({'逛街': 2})


In [39]:
Counter([j for i in data_tagged['Style_Category'] for j in i])

Counter({'未明确提及': 1574,
         '可爱': 371,
         '时尚': 278,
         '复古': 194,
         '好看': 184,
         '独特': 134,
         '高级感': 118,
         '简洁': 89,
         '百搭': 80,
         '千禧年': 56,
         '其他': 53,
         '清新': 2})

In [40]:
from collections import Counter
def count_word_freq(column):
    # Flatten all lists from the column into a single list, ignoring NaN values
    all_keywords = [word for keywords in column for word in keywords]
    
    # Count the frequency of each word using Counter
    return Counter(all_keywords)
# Function to get top N frequent keywords
def get_top_n(counter_obj, n=5):
    return counter_obj.most_common(n)
get_top_n(count_word_freq(data_tagged['Style_Category']))

[('未明确提及', 1383), ('可爱', 399), ('时尚', 298), ('好看', 262), ('复古', 219)]

In [1]:
data_tagged.columns

NameError: name 'data_tagged' is not defined